In [18]:
import pandas as pd
import os
import json

path = r"C:\Users\Admin\KIENQUOC\UIT\2025-2026\bflow\chatbot\mis_ai\statics\accounting_docs\account.xlsx"

if not os.path.exists(path):
    print("Không tìm thấy file:", path)
else:
    df = pd.read_excel(path, header=0)

    df = df.dropna(how='all')
    df.columns = df.columns.str.strip()

    df = df.fillna("")
    data_json = df.to_dict(orient='records')

    output_path = r"C:\Users\Admin\KIENQUOC\UIT\2025-2026\bflow\chatbot\mis_ai\statics\accounting_docs\account.json"
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data_json, f, ensure_ascii=False, indent=4)

    print(f"Đã chuyển thành JSON và lưu tại:\n{output_path}")


Đã chuyển thành JSON và lưu tại:
C:\Users\Admin\KIENQUOC\UIT\2025-2026\bflow\chatbot\mis_ai\statics\accounting_docs\account.json


In [23]:
import json

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

path = r"C:\Users\Admin\KIENQUOC\UIT\2025-2026\bflow\chatbot\mis_ai\statics\accounting_docs\account.json"

with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(data)

print("Đang load model SentenceTransformer...")
_model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')
print("Model loaded thành công!")

_texts = [
    f"{d['AccountNumber']} {d['VietnameseName']} {d['EnglishName']} {d['Meaning']} {d['AccountingMethod']} {d['Usage']} {d['CorrespondingAccounts']}"
    for d in data]
_embeddings = _model.encode(_texts)
_dimension = _embeddings.shape[1]
_index = faiss.IndexFlatL2(_dimension)
_index.add(np.array(_embeddings).astype('float32'))
print("FAISS index đã sẵn sàng cho features!")


def rag_accounting(user_input, top_k=1):
    try:
        user_embedding = _model.encode([user_input])
        D, I = _index.search(np.array(user_embedding).astype('float32'), k=top_k)
        print(user_input)
        results = []
        for idx, dist in zip(I[0], D[0]):
            da = data[idx]
            results.append({
                "AccountNumber": da["AccountNumber"],
                "VietnameseName": da["VietnameseName"],
                "EnglishName": da["EnglishName"],
                "Meaning": da["Meaning"],
                "AccountingMethod": da["AccountingMethod"],
                "Usage": da["Usage"],
                "CorrespondingAccounts": da["CorrespondingAccounts"],

                "distance": float(dist)
            })

        for result in results:
            print(result)
        return results[0] if results else None

    except Exception as e:
        print("Lỗi RAG:", e)
        return None

rag_accounting('Tôi muốn tài khoản ghi tiền mặt', top_k=1)

C:\Users\Admin\KIENQUOC\UIT\2025-2026\bflow\chatbot\mis_ai\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'AccountNumber': 111, 'VietnameseName': 'Tiền mặt', 'EnglishName': 'Cash on hand', 'Meaning': 'Phản ánh số hiện có và tình hình biến động của tiền mặt tại quỹ (VND, Ngoại tệ, Vàng tiền tệ).', 'AccountingMethod': 'Tăng ghi Nợ, giảm ghi Có (Tài sản)', 'Usage': 'Mọi nghiệp vụ thu, chi bằng tiền mặt tại quỹ.', 'CorrespondingAccounts': 'Thu tiền: Nợ 111 / Có 112 (Rút tiền NH), 131 (Thu nợ), 511, 711 (Thu DT/TN khác). Chi tiền: Nợ 112 (Nộp vào NH), 141 (Tạm ứng), 331 (Trả nợ), 641, 642 (Chi phí).'}, {'AccountNumber': 1111, 'VietnameseName': 'Tiền Việt Nam', 'EnglishName': 'Vietnamese Dong', 'Meaning': 'Tiền Việt Nam hiện có tại quỹ.', 'AccountingMethod': 'Tăng ghi Nợ, giảm ghi Có', 'Usage': 'Thu, chi bằng VND.', 'CorrespondingAccounts': 'Nghiệp vụ chi tiết TK 111 (chỉ liên quan VND).'}, {'AccountNumber': 1112, 'VietnameseName': 'Ngoại tệ', 'EnglishName': 'Foreign Currency', 'Meaning': 'Ngoại tệ hiện có tại quỹ.', 'AccountingMethod': 'Tăng ghi Nợ, giảm ghi Có', 'Usage': 'Thu, chi bằng ngoại

{'AccountNumber': 111,
 'VietnameseName': 'Tiền mặt',
 'EnglishName': 'Cash on hand',
 'Meaning': 'Phản ánh số hiện có và tình hình biến động của tiền mặt tại quỹ (VND, Ngoại tệ, Vàng tiền tệ).',
 'AccountingMethod': 'Tăng ghi Nợ, giảm ghi Có (Tài sản)',
 'Usage': 'Mọi nghiệp vụ thu, chi bằng tiền mặt tại quỹ.',
 'CorrespondingAccounts': 'Thu tiền: Nợ 111 / Có 112 (Rút tiền NH), 131 (Thu nợ), 511, 711 (Thu DT/TN khác). Chi tiền: Nợ 112 (Nộp vào NH), 141 (Tạm ứng), 331 (Trả nợ), 641, 642 (Chi phí).',
 'distance': 40.6485595703125}